<a href="https://colab.research.google.com/github/RonaldTuna/TensorIoTAssignment/blob/main/TensorIoT_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**STEPS:**

Download the data files from here - http://jmcauley.ucsd.edu/data/amazon/links.html

Apache spark tools locally and necessary tools

Download a review file with a million reviews

Using Jupyter notebook create a program to read the million reviews and get the following

transform date to MM-DD-YYYY format

Save the data into a table (postgres/sql server)
Save the output as a Parquet file


Upload code to Github  and complete Readme.md which anyone can understand

Send Github link to HR



Skills Learning / Tools Used:

    postgres
    workbench/J
    Jupyter notebook
    Apache spark
    Data frame transforms
    









In [1]:
# Install Java, Spark, Findspark and download a Postgresql driver
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q -nc https://downloads.apache.org/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz
!tar xf spark-3.2.1-bin-hadoop3.2.tgz
!pip install -q findspark

!wget -nc https://jdbc.postgresql.org/download/postgresql-42.3.5.jar

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.2.1-bin-hadoop3.2"

# Start a SparkSession
import findspark
findspark.init()

--2022-05-13 02:57:13--  https://jdbc.postgresql.org/download/postgresql-42.3.5.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1041066 (1017K) [application/java-archive]
Saving to: ‘postgresql-42.3.5.jar’

postgresql-42.3.5.j 100%[===================>]   1017K  5.14MB/s    in 0.2s    

2022-05-13 02:57:14 (5.14 MB/s) - ‘postgresql-42.3.5.jar’ saved [1041066/1041066]



In [2]:
#install pyspark
try:
  %pyspark_version 3.x
except Exception:
  pass

!pip install pyspark

In [3]:
# Install postgresql server
!sudo apt-get -y -qq update
!sudo apt-get -y -qq install postgresql
!sudo service postgresql start

# Setup a password `postgres` for username `postgres`
!sudo -u postgres psql -U postgres -c "ALTER USER postgres PASSWORD 'postgres';"

# Setup a database with name `reviews` to be used
!sudo -u postgres psql -U postgres -c 'DROP DATABASE IF EXISTS reviews;'
!sudo -u postgres psql -U postgres -c 'CREATE DATABASE reviews;'

 * Starting PostgreSQL 10 database server
   ...done.
ALTER ROLE
DROP DATABASE
CREATE DATABASE


In [4]:
#setting up environment
%env REVIEWS_DATABASE_NAME=reviews
%env REVIEWS_DATABASE_HOST=localhost
%env REVIEWS_DATABASE_PORT=5432 
%env REVIEWS_DATABASE_USER=postgres
%env REVIEWS_DATABASE_PASS=postgres

env: REVIEWS_DATABASE_NAME=reviews
env: REVIEWS_DATABASE_HOST=localhost
env: REVIEWS_DATABASE_PORT=5432
env: REVIEWS_DATABASE_USER=postgres
env: REVIEWS_DATABASE_PASS=postgres


In [5]:
#import pyspark and required tools
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

I extracted the reviews for Android Applications, which contains 2,638,173 reviews using wget.

I then load the reviews into a pyspark dataframe

In [6]:
#download review file from link 
!wget -nc "http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Apps_for_Android.json.gz"

#create a spark session
spark = SparkSession \
    .builder \
    .config("spark.jars", "/content/spark-3.2.1-bin-hadoop3.2/jars/postgresql-42.3.5.jar")\
    .appName("AmazonReviews") \
    .getOrCreate()
df = spark.read.json('/content/reviews_Apps_for_Android.json.gz')

--2022-05-13 02:57:29--  http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Apps_for_Android.json.gz
Resolving snap.stanford.edu (snap.stanford.edu)... 171.64.75.80
Connecting to snap.stanford.edu (snap.stanford.edu)|171.64.75.80|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 299986290 (286M) [application/x-gzip]
Saving to: ‘reviews_Apps_for_Android.json.gz’

reviews_Apps_for_An 100%[===================>] 286.09M  41.1MB/s    in 8.0s    

2022-05-13 02:58:16 (35.6 MB/s) - ‘reviews_Apps_for_Android.json.gz’ saved [299986290/299986290]



Transform data from MM DD, YYYY to MM-DD-YYYY by using to_date() to turn the string into a date object, and used date_format() to reformat to specified format.

Replace old column with new column containing reformatted dates by deleting the old column and renaming the new one.

In [7]:
df = df.withColumn('newTime', date_format(to_date('reviewTime', format='MM d, yyyy'), "MM-dd-yyyy"))
df = df.drop("reviewTime")
df = df.withColumnRenamed("newTime","reviewTime")
df.show()

+----------+--------+-------+--------------------+--------------+--------------------+--------------------+--------------+----------+
|      asin| helpful|overall|          reviewText|    reviewerID|        reviewerName|             summary|unixReviewTime|reviewTime|
+----------+--------+-------+--------------------+--------------+--------------------+--------------------+--------------+----------+
|B004A9SDD8|  [0, 0]|    5.0|Glad to finally s...| AUI0OLXAB3KKT|          A Customer|        Great app!!!|    1301184000|03-27-2011|
|B004A9SDD8|[12, 14]|    5.0|this app works gr...|A1ZUSQ3TC3EC4C|           A. Lissak|        Kid loves it|    1321574400|11-18-2011|
|B004A9SDD8|  [0, 0]|    4.0|We love these mon...| AC05OAXD72X1V|               Allie| Love these monkeys!|    1367366400|05-01-2013|
|B004A9SDD8|  [0, 2]|    5.0|cannot get my kin...|A2RVMFOKBVM21I|     Amazon Customer|fun fun for toddlers|    1350172800|10-14-2012|
|B004A9SDD8|  [1, 3]|    1.0|I start this app ...|A3NBSRGUWQGC

Load data into SQL table in local postgreSQL server

In [8]:
mode = "overwrite"
jdbc_url="jdbc:postgresql://localhost:5432/reviews"
config = {"user":"postgres",
          "password":"postgres",  
          "driver":"org.postgresql.Driver"}
#NOT USED

In [9]:
#writing to dataframe to SQL server
df.write \
    .format("jdbc") \
    .mode("overwrite") \
    .option("url", "jdbc:postgresql://localhost:5432/reviews") \
    .option("dbtable", "reviews") \
    .option("user","postgres") \
    .option("password","postgres") \
    .option("driver", "org.postgresql.Driver") \
    .save()

Py4JJavaError: ignored

In [ ]:
df.write.jdbc(url=jdbc_url,
              table ="reviews", 
              mode=mode, properties = config)

Save dataframe as parquet file.

In [ ]:
df.write.parquet("/content/Amazon_reviews.parquet")